**Curso de Inteligencia Artificial y Aprendizaje Profundo**


# Redes Gated Recurrent Unit-GRU

##  Autores

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
2. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 
3. Oleg Jarma, ojarmam@unal.edu.co
4. Maria del Pilar Montenegro, pmontenegro88@gmail.com

## Contenido

* [Introducción](#Introducción)
* [Funcionamiento de una red GRU](#Funcionamiento-de-una-red-GRU)
* [Puerta de actualización-update](#Puerta-de-actualización-update)
* [Puerta reinicio](#Puerta-reinicio)
* [Activación candidata](#Activación-candidata)
* [Actualización del estado recurrente](#Actualización-del-estado-recurrente)


## Referencias

1.  Cho et al. 2014 [On the Properties of Neural Machine Translation: Encoder–Decoder
Approaches](https://arxiv.org/pdf/1409.1259.pdf)
2. J. Chung, C. Gulcehre, K. Cho, Y. Bengio, [Empirical Evaluation of Gated Recurrent Neural Networks on Sequence  Modeling](https://arxiv.org/pdf/1412.3555v1.pdf)
3. Karpathy, [*The Unreasonable Effectiveness of Recurrent Neural Networks*](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)



## Introducción



Una red neuronal recurrente (RNR) es una extensión de una red neuronal convencional,
que es capaz de manejar una entrada de secuencia de longitud variable. La RNR maneja la longitud variable de las secuencias mediante un estado oculto recurrente cuya activación en cada momento depende del estado anterior. 

Más formalmente, dada una secuencia $\mathbf{x}= (x_1, x_2, \ldots, x_T)$, la RNR actualiza su estado oculto recurrente $h_t$ mediante 


$$
h_t = \begin{cases}
0,  &\text{ si } t=0,\\
\phi(h_{t-1},x_t), &\text{en otro caso}.
\end{cases}
$$


Si $g$ es una función de activación  suave, como un sigmoide o una tangente hiperbólica, es común definir

$$
h_t = g(Wh_{t-1} + Ux_t+ b)
$$

Una RNR generativa genera una distribución de probabilidad sobre el siguiente elemento de la secuencia, dado su estado actual $h_t$, y este modelo generativo puede capturar una distribución sobre secuencias de longitud variable mediante el uso de un símbolo de salida especial para representar el final de la secuencia. La secuencia la probabilidad se puede descomponer en

$$
p (x_1,\ldots, x_T) = p (x_1) p (x_2 | x_1) p (x_3 | x_1, x_2) \ldots p (x_T | x_1,\ldots, x_{T − 1}), 
$$

donde el último elemento es un valor especial de final de secuencia. Modelamos cada probabilidad condicional distribución con

$$
p (x_t | x_1,\ldots, x_{t − 1}) = g (h_t)
$$

El problema con este modelo, es que el cálculo del gradiente tiende a volverse cero o a explotar. 

Dos líneas de trabajo impulsó esta situación. Por un lado se inició la búsqueda de nuevas técnicas para para el uso del gradiente en el proceso de optimización de  la función de costo y por el otro, el desarrollo de nuevos modelos de redes neuronales. 


La primera línea ha producido nuevas técnicas de optimización estocástica basados en el gradiente, que han sido usado exitosamente en redes generales. 

La segunda línea llevó al desarrollo de las redes [LSTM](Intro_LSTM.ipynb), en las cuales la función de activación consiste en una transformación afinada seguida por una simple no linealidad de elementos mediante el uso de unidades de compuerta, [Hochreiter y Schmidhuber, 1997](https://www.bioinf.jku.at/publications/older/2604.pdf). 


Más recientemente, otro tipo de unidad recurrente, a la que nos referimos como una unidad recurrente cerrada (GRU), fue propuesta por [Cho et al. 2014](https://arxiv.org/pdf/1409.1259.pdf). Puede consultar una comparación entre LSTM y GRU en [Junyoung Chung et al., 2014](https://arxiv.org/pdf/1412.3555v1.pdf). De estas unidades recurrentes se ha demostrado que funcionan bien en tareas que requieren captura de dependencias a largo plazo.  Esas tareas incluyen, pero no se limitan a reconocimiento de voz, música,...

## Funcionamiento de una red GRU

Una red de unidad recurrente cerrada (GRU) permite que cada unidad recurrente capture de forma adaptativa dependencias de diferentes escalas de tiempo. De manera similar a la unidad LSTM, la GRU tiene puertas que modulan el flujo de información dentro de la unidad.  Sin embargo, a diferencia de las redes LSTM no tiene celdas de memoria separadas.

El diagrama presenta la estructura general de una compuerta en una red GRU. A la entrada de cada unidad se presenta, la información de memoria procedente de la unida anterior y la información en la unidad de tiempo $t$. 


<figure>
<center>
<img src="../Imagenes/compuerta_GRU_after_false.jpg" width="800" height="600" align="center"/>
</center>
</figure>

Imagen: Alvaro Montenegro

reset_after: GRU convention (whether to apply reset gate after or before matrix multiplication). False = "before", True = "after" (default and CuDNN compatible).

<figure>
<center>
<img src="../Imagenes/compuerta_GRU_after_true.jpg" width="800" height="600" align="center"/>
</center>
</figure>

Imagen: Alvaro Montenegro

La notación que usamos es bastante estándar. 

1. $+$ : indica suma de vectores
2. $\sigma$ : representa a la función de activación sigmoide
3. $\tanh$ : representa a la función de activación tangente hiperbólica.
4.$\odot$ : es producto componente a componente (producto de Hamard)

Una GRU tien dos tipos de puerta: actualización (update) y reinicio (reset).

## Puerta de actualización-update




Comenzamos calculando la puerta de actualización $z_t$ para el paso de tiempo $t$ usando la fórmula:

$$
z_t = \sigma(W_z x_t + U_zh_{t-1} + b_z)
$$

en donde $W_z$ y $U_z$ son pesos asociados $x_t$ (la nueva entrada) y $h_{t-1}$, (la  información procedente de la unidad anterior).


La puerta de actualización ayuda al modelo a determinar qué cantidad de la información pasada (de los pasos de tiempo anteriores) debe transmitirse al futuro, combinandola  con la nueva información. 


Eso es realmente poderoso porque el modelo puede decidir copiar toda la información del pasado y eliminar el riesgo de desvanecer del gradiente.

## Puerta reinicio

Esencialmente, esta puerta se utiliza para que el modelo decida qué cantidad de información pasada debe olvidar. Para calcularla, utilizamos:

$$
r_t = \sigma(W_r x_t + U_rh_{t-1} + b_r),
$$

en donde $W_r$ y $U_r$ son pesos asociados $x_t$ (la nueva entrada) y $h_{t-1}$, (la  información procedente de la unidad anterior).

## Activación candidata



Veamos cómo afectarán exactamente las puertas al resultado final. Primero, comenzamos con el uso de la puerta de reinicio. Introducimos un nuevo contenido de memoria que utilizará la puerta de reinicio para almacenar la información relevante del pasado. Se calcula de la siguiente manera:


$$
\tilde{h}_t = \tanh(Wx_t + U(r_t\odot h_{t-1})+b),
$$

en donde $W$ y $U$ son pesos asociados a las  $x$'s y a las $h$'s respectivamente.

## Actualización del estado recurrente

La activación de la GRU en tiempo $t$ es una interpolación lineal entre la activación previa ${h}_{t-1}$ y la activación candidata $\tilde{h}_t$ definida arriba. 

En símbolos tenemos:


$$
h_t = (1-z_t)\odot h_{t-1} + z_t \odot \tilde{h}_t 
$$

La siguiente imagen ilustra la arquitectura de la red.

<figure>
<center>
<img src="../Imagenes/plano_gru.jpg" width="800" height="600" align="center"/>
</center>
</figure>

Imagen: Alvaro Montenegro

[Regresar al inicio](#Contenido)